In [1]:
import numpy as np
import torch

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [2]:
torch.__version__

'2.9.1'

In [3]:
import torchvision
print(torchvision.__version__)

0.24.1


In [4]:
import torch.nn as nn
import torch.nn.functional as F

In [19]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, (3, 3))
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


model = Net()
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

In [20]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
         MaxPool2d-2           [-1, 32, 99, 99]               0
            Linear-3                   [-1, 64]      20,072,512
            Linear-4                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 11.96
Params size (MB): 76.57
Estimated Total Size (MB): 89.00
----------------------------------------------------------------
Total parameters: 20073473


In [21]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize((200, 200)),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [22]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

train_dataset = torchvision.datasets.ImageFolder(
    root='data/train',
    transform=train_transforms
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=2
)

validation_dataset = torchvision.datasets.ImageFolder(
    root='data/test',
    transform=train_transforms
)
validation_loader = torch.utils.data.DataLoader(
    validation_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=2
)

criterion = nn.BCEWithLogitsLoss()

In [11]:
device

device(type='cpu')

In [12]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 800
    Root location: data/train
    StandardTransform
Transform: Compose(
               Resize(size=(200, 200), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [23]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0) # * images.size(0) to get total loss for the batch
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).float().sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0) # * images.size(0) to get total loss for the batch
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).float().sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6455, Acc: 0.6375, Val Loss: 0.6026, Val Acc: 0.6716
Epoch 2/10, Loss: 0.5715, Acc: 0.7113, Val Loss: 0.5961, Val Acc: 0.6468
Epoch 3/10, Loss: 0.5577, Acc: 0.7050, Val Loss: 0.5880, Val Acc: 0.6617
Epoch 4/10, Loss: 0.4985, Acc: 0.7512, Val Loss: 0.5863, Val Acc: 0.6716
Epoch 5/10, Loss: 0.4299, Acc: 0.7975, Val Loss: 0.5748, Val Acc: 0.6915
Epoch 6/10, Loss: 0.3880, Acc: 0.8325, Val Loss: 0.8449, Val Acc: 0.6219
Epoch 7/10, Loss: 0.3759, Acc: 0.8413, Val Loss: 0.6057, Val Acc: 0.7065
Epoch 8/10, Loss: 0.3115, Acc: 0.8775, Val Loss: 0.6719, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2918, Acc: 0.8762, Val Loss: 0.6145, Val Acc: 0.7114
Epoch 10/10, Loss: 0.1716, Acc: 0.9550, Val Loss: 0.6689, Val Acc: 0.6866


poch 1/10, Loss: 0.6455, Acc: 0.6375, Val Loss: 0.6026, Val Acc: 0.6716
Epoch 2/10, Loss: 0.5715, Acc: 0.7113, Val Loss: 0.5961, Val Acc: 0.6468
Epoch 3/10, Loss: 0.5577, Acc: 0.7050, Val Loss: 0.5880, Val Acc: 0.6617
Epoch 4/10, Loss: 0.4985, Acc: 0.7512, Val Loss: 0.5863, Val Acc: 0.6716
Epoch 5/10, Loss: 0.4299, Acc: 0.7975, Val Loss: 0.5748, Val Acc: 0.6915
Epoch 6/10, Loss: 0.3880, Acc: 0.8325, Val Loss: 0.8449, Val Acc: 0.6219
Epoch 7/10, Loss: 0.3759, Acc: 0.8413, Val Loss: 0.6057, Val Acc: 0.7065
Epoch 8/10, Loss: 0.3115, Acc: 0.8775, Val Loss: 0.6719, Val Acc: 0.7214
Epoch 9/10, Loss: 0.2918, Acc: 0.8762, Val Loss: 0.6145, Val Acc: 0.7114
Epoch 10/10, Loss: 0.1716, Acc: 0.9550, Val Loss: 0.6689, Val Acc: 0.6866

In [ ]:
# std deviation of training loss
train_loss_std = np.std(history['loss'])

In [ ]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(50),
    torchvision.transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [ ]:
# Resume training for 10 more epochs
num_epochs_resume = 10
for epoch in range(num_epochs_resume):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += torch.as_tensor(predicted == labels).float().sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += torch.as_tensor(predicted == labels).float().sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"[Resume] Epoch {epoch+1}/{num_epochs_resume}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

In [ ]:
# Resume training for 10 more epochs with augmented data
def resume_training(model, optimizer, train_loader, validation_loader, criterion, history, device, num_epochs=10):
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        correct_train = 0
        total_train = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_train += labels.size(0)
            correct_train += torch.as_tensor(predicted == labels).float().sum().item()

        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = correct_train / total_train
        history['loss'].append(epoch_loss)
        history['acc'].append(epoch_acc)

        model.eval()
        val_running_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for images, labels in validation_loader:
                images, labels = images.to(device), labels.to(device)
                labels = labels.float().unsqueeze(1)

                outputs = model(images)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item() * images.size(0)
                predicted = (torch.sigmoid(outputs) > 0.5).float()
                total_val += labels.size(0)
                correct_val += torch.as_tensor(predicted == labels).float().sum().item()

        val_epoch_loss = val_running_loss / len(validation_loader.dataset)
        val_epoch_acc = correct_val / total_val
        history['val_loss'].append(val_epoch_loss)
        history['val_acc'].append(val_epoch_acc)

        print(f"[Resume] Epoch {epoch+1}/{num_epochs}, "
              f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
              f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

# Call the function to resume training
resume_training(model, optimizer, train_loader, validation_loader, criterion, history, device, num_epochs=10)

In [ ]:
# Optionally, plot the training and validation loss/accuracy curves
import matplotlib.pyplot as plt
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(history['loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.subplot(1,2,2)
plt.plot(history['acc'], label='Train Acc')
plt.plot(history['val_acc'], label='Val Acc')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()
